In [167]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 100
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

from sklearn.model_selection import train_test_split   # разделение выборки на train и test

from sklearn.model_selection import cross_val_score     # кросс-валидация
from sklearn.model_selection import cross_val_predict

from sklearn.linear_model import LogisticRegression
import xgboost as xgb, lightgbm as lgbm, catboost as catb

from sklearn.metrics import roc_auc_score

In [168]:
# https://www.kaggle.com/c/geekbrains-competitive-data-analysis/overview

In [169]:
# пути
path_train = '/home/tefi/Tefi/Project_sport/train.csv'
path_test = '/home/tefi/Tefi/Project_sport/test.csv'
path_applications_history = '/home/tefi/Tefi/Project_sport/applications_history.csv'
path_bki = '/home/tefi/Tefi/Project_sport/bki.csv'
path_client_profile = '/home/tefi/Tefi/Project_sport/client_profile.csv'
path_payments = '/home/tefi/Tefi/Project_sport/payments.csv'
path_sample_submit = '/home/tefi/Tefi/Project_sport/sample_submit.csv'

# загрузка данных из файлов в соответствующие переменные
train_pro = pd.read_csv(path_train)
test_pro = pd.read_csv(path_test)
applications_history = pd.read_csv(path_applications_history)
bki = pd.read_csv(path_bki)
client_profile = pd.read_csv(path_client_profile)
payments = pd.read_csv(path_payments)
sample_submit = pd.read_csv(path_sample_submit)  # образец submit

Описание датасета:

APPLICATION_NUMBER - номер заявки
NAME_CONTRACT_TYPE - тип договора (наличные, карта)
TARGET - целевая переменная

История предыдущих заявок клиента:
PREV_APPLICATION_NUMBER - предыдущий номер заявки
AMOUNT_ANNUITY - сумма в год
AMT_APPLICATION - сумма заявки
AMOUNT_CREDIT - сумма кредита
AMOUNT_PAYMENT - сумма выплаты
AMOUNT_GOODS_PAYMENT - сумма стоимости товара 
NAME_CONTRACT_STATUS - статус годовора (одобрено, отклонено)
DAYS_DECISION - разрешение на кол-во дней
NAME_PAYMENT_TYPE - тип оплаты (наличными через банк и т.п.)
CODE_REJECT_REASON - причина отклонения
NAME_TYPE_SUITE - поручители
NAME_CLIENT_TYPE - тип клиента (новый, повторно)
NAME_GOODS_CATEGORY - категория товара
NAME_PORTFOLIO - название портфеля (наличные, карта)
NAME_PRODUCT_TYPE - тип товара (x-sell, wolk-in)
SELLERPLACE_AREA - площадь продажи
CNT_PAYMENT - кол-во выплат по кредиту
NAME_YIELD_GROUP - доходная группа (средняя, ниже среднего)
DAYS_FIRST_DRAWING - 
DAYS_FIRST_DUE - первый срок
DAYS_LAST_DUE_1ST_VERSION - прошло дней с 1 версии
DAYS_LAST_DUE - последний день
DAYS_TERMINATION - дней до завершения
NFLAG_INSURED_ON_APPROVAL - застрахован по принуждению

данные БКИ о предыдущих кредитах клиента:
BUREAU_ID - номер бюро
CREDIT_ACTIVE - активен ли кредит
CREDIT_CURRENCY - валюта кредита
DAYS_CREDIT - дни кредита
CREDIT_DAY_OVERDUE - кол-во дней просрочки
DAYS_CREDIT_ENDDATE - дней до конца кредита
DAYS_ENDDATE_FACT - фактически дней
AMT_CREDIT_MAX_OVERDUE - максимальная сумма просроченного кредита
CNT_CREDIT_PROLONG - продление текущего кредита
AMT_CREDIT_SUM - сумма кредита
AMT_CREDIT_SUM_DEBT - задолженность по кредиту
AMT_CREDIT_SUM_LIMIT - сумма предельного кредита
AMT_CREDIT_SUM_OVERDUE - сумма просроченного кредита
CREDIT_TYPE - тип кредита (потребительский)
DAYS_CREDIT_UPDATE - обновление дней кредита
AMT_ANNUITY - сумма ежегодной выплаты

Клиентский профиль, некоторые знания, которые есть у компании о клиенте:
GENDER - пол
CHILDRENS - дети
TOTAL_SALARY - зарплата
AMOUNT_CREDIT - сумма кредита
AMOUNT_ANNUITY - сумма ежегодной выплаты
EDUCATION_LEVEL - образование
FAMILY_STATUS - семейный статус (женат)
REGION_POPULATION - население региона
AGE - возраст
DAYS_ON_LAST_JOB - дней на последней работе 
OWN_CAR_AGE - возраст собственного автомобиля
FLAG_PHONE - наличие телефона
FLAG_EMAIL - наличие электронной почты
FAMILY_SIZE - размер семьи (2, 3)
EXTERNAL_SCORING_RATING_1 - внешний рейтинг 1
EXTERNAL_SCORING_RATING_2 - внешний рейтинг 2
EXTERNAL_SCORING_RATING_3 - внешний рейтинг 3
AMT_REQ_CREDIT_BUREAU_HOUR - сумма запросов на выдачу кредитов
AMT_REQ_CREDIT_BUREAU_DAY - сумма запросов на выдачу кредитов
AMT_REQ_CREDIT_BUREAU_WEEK - сумма запросов на выдачу кредитов
AMT_REQ_CREDIT_BUREAU_MON - сумма запросов на выдачу кредитов
AMT_REQ_CREDIT_BUREAU_QRT - сумма запросов на выдачу кредитов
AMT_REQ_CREDIT_BUREAU_YEAR - сумма запросов на выдачу кредитов

История платежей клиента:
NUM_INSTALMENT_VERSION - номер расчетной версии
NUM_INSTALMENT_NUMBER - расчетный номер
DAYS_INSTALMENT - дни рассрочки
DAYS_ENTRY_PAYMENT - оплата по кредиту в день
AMT_INSTALMENT - сумма расчета
AMT_PAYMENT - сумма выплаты

In [170]:
train_pro.head(5)

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE
0,123687442,0,Cash
1,123597908,1,Cash
2,123526683,0,Cash
3,123710391,1,Cash
4,123590329,1,Cash


In [171]:
train_pro['APPLICATION_NUMBER'].nunique()==train_pro.shape[0]  

True

In [172]:
applications_history['APPLICATION_NUMBER'].nunique()==applications_history.shape[0] 

False

In [173]:
bki['APPLICATION_NUMBER'].nunique()==bki.shape[0]

False

In [174]:
client_profile['APPLICATION_NUMBER'].nunique()==client_profile.shape[0]

True

In [175]:
payments['APPLICATION_NUMBER'].nunique()==payments.shape[0]

False

In [176]:
train = pd.merge(train_pro, client_profile, on='APPLICATION_NUMBER', how='left')
train_data = train.drop(['TARGET'], axis = 'columns')
train_target = train['TARGET']
train_data

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123687442,Cash,M,1.0,157500.0,855000.0,25128.0,Secondary / secondary special,Married,0.019101,15728.0,1719.0,11.0,0.0,0.0,3.0,0.700784,0.645914,0.716570,0.0,0.0,1.0,0.0,0.0,2.0
1,123597908,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,123526683,Cash,F,0.0,135000.0,1006920.0,42660.0,Higher education,Married,0.026392,21557.0,3618.0,NaN,1.0,0.0,2.0,NaN,0.682149,0.267869,0.0,0.0,0.0,7.0,0.0,4.0
3,123710391,Cash,M,0.0,180000.0,518562.0,22972.5,Secondary / secondary special,Married,0.031329,22338.0,365243.0,NaN,0.0,0.0,2.0,NaN,0.171299,0.170446,0.0,0.0,0.0,0.0,0.0,0.0
4,123590329,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110088,123458312,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110089,123672463,Cash,F,0.0,175500.0,269550.0,12618.0,Secondary / secondary special,Married,0.010032,10519.0,1354.0,NaN,0.0,0.0,2.0,0.528096,0.471774,0.581484,0.0,0.0,0.0,0.0,0.0,2.0
110090,123723001,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110091,123554358,Cash,F,2.0,270000.0,1024740.0,49428.0,Incomplete higher,Married,0.019101,15008.0,361.0,22.0,0.0,0.0,4.0,0.653826,0.651227,0.558507,0.0,0.0,0.0,1.0,0.0,1.0


In [177]:
test_data = pd.merge(test_pro, client_profile, on='APPLICATION_NUMBER', how='left')
test_data.head(2)

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123724268,Cash,M,0.0,117000.0,1125000.0,32895.0,Secondary / secondary special,Married,0.028663,16007.0,2646.0,20.0,0.0,0.0,2.0,NaN,0.628266,NaN,0.0,0.0,0.0,0.0,1.0,4.0
1,123456549,Cash,F,2.0,81000.0,312768.0,17095.5,Secondary / secondary special,Married,0.019689,10315.0,459.0,NaN,0.0,0.0,4.0,NaN,0.578161,0.18849,0.0,0.0,1.0,0.0,0.0,2.0


In [178]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110093 entries, 0 to 110092
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   APPLICATION_NUMBER          110093 non-null  int64  
 1   NAME_CONTRACT_TYPE          110093 non-null  object 
 2   GENDER                      89539 non-null   object 
 3   CHILDRENS                   89539 non-null   float64
 4   TOTAL_SALARY                89539 non-null   float64
 5   AMOUNT_CREDIT               89539 non-null   float64
 6   AMOUNT_ANNUITY              89534 non-null   float64
 7   EDUCATION_LEVEL             89539 non-null   object 
 8   FAMILY_STATUS               89539 non-null   object 
 9   REGION_POPULATION           89539 non-null   float64
 10  AGE                         89539 non-null   float64
 11  DAYS_ON_LAST_JOB            89539 non-null   float64
 12  OWN_CAR_AGE                 30533 non-null   float64
 13  FLAG_PHONE    

In [179]:
features_1 = ['GENDER', 'CHILDRENS', 'EDUCATION_LEVEL', 'FAMILY_STATUS', 'REGION_POPULATION', 'OWN_CAR_AGE', 
             'FLAG_PHONE', 'FLAG_EMAIL', 'FAMILY_SIZE', 'EXTERNAL_SCORING_RATING_1', 'EXTERNAL_SCORING_RATING_2', 
              'EXTERNAL_SCORING_RATING_3', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 
              'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 
              'AMT_REQ_CREDIT_BUREAU_YEAR']

for i in features_1:
    train_data.loc[(train_data[i].isnull(), [i])] = train_data[i].mode()[0]
    
for i in features_1:
    test_data.loc[(test_data[i].isnull(), [i])] = train_data[i].mode()[0]

In [180]:
features_2 = ['TOTAL_SALARY', 'AMOUNT_CREDIT', 'AMOUNT_ANNUITY', 'AGE', 'DAYS_ON_LAST_JOB']

for i in features_2:
    train_data.loc[(train_data[i].isnull(), [i])] = train_data[i].median()
for i in features_2:
    test_data.loc[(test_data[i].isnull(), [i])] = train_data[i].median()

In [181]:
train_data.select_dtypes(include = ['float64']).columns

Index(['CHILDRENS', 'TOTAL_SALARY', 'AMOUNT_CREDIT', 'AMOUNT_ANNUITY',
       'REGION_POPULATION', 'AGE', 'DAYS_ON_LAST_JOB', 'OWN_CAR_AGE',
       'FLAG_PHONE', 'FLAG_EMAIL', 'FAMILY_SIZE', 'EXTERNAL_SCORING_RATING_1',
       'EXTERNAL_SCORING_RATING_2', 'EXTERNAL_SCORING_RATING_3',
       'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
       'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
       'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object')

In [182]:
features_3 = ['CHILDRENS', 'TOTAL_SALARY', 'AMOUNT_CREDIT', 'AMOUNT_ANNUITY', 'AGE', 'DAYS_ON_LAST_JOB', 
              'OWN_CAR_AGE', 'FLAG_PHONE', 'FLAG_EMAIL', 'FAMILY_SIZE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 
              'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
              'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

for i in features_3:
    train_data[i] = train_data[i].astype(int)
for i in features_3:
    test_data[i] = test_data[i].astype(int)

In [183]:
train_data.select_dtypes(include = ['object']).columns

Index(['NAME_CONTRACT_TYPE', 'GENDER', 'EDUCATION_LEVEL', 'FAMILY_STATUS'], dtype='object')

In [184]:
features_4 = ['NAME_CONTRACT_TYPE', 'GENDER', 'EDUCATION_LEVEL', 'FAMILY_STATUS']
for i in features_4:
    train_data = pd.concat([train_data, pd.get_dummies(train_data[i])], axis=1)
for i in features_4:
    test_data = pd.concat([test_data, pd.get_dummies(test_data[i])], axis=1)

train_data = train_data.drop(['NAME_CONTRACT_TYPE', 'GENDER', 'EDUCATION_LEVEL', 'FAMILY_STATUS'], axis = 'columns')
test_data = test_data.drop(['NAME_CONTRACT_TYPE', 'GENDER', 'EDUCATION_LEVEL', 'FAMILY_STATUS'], axis = 'columns')


In [185]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110093 entries, 0 to 110092
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   APPLICATION_NUMBER             110093 non-null  int64  
 1   CHILDRENS                      110093 non-null  int64  
 2   TOTAL_SALARY                   110093 non-null  int64  
 3   AMOUNT_CREDIT                  110093 non-null  int64  
 4   AMOUNT_ANNUITY                 110093 non-null  int64  
 5   REGION_POPULATION              110093 non-null  float64
 6   AGE                            110093 non-null  int64  
 7   DAYS_ON_LAST_JOB               110093 non-null  int64  
 8   OWN_CAR_AGE                    110093 non-null  int64  
 9   FLAG_PHONE                     110093 non-null  int64  
 10  FLAG_EMAIL                     110093 non-null  int64  
 11  FAMILY_SIZE                    110093 non-null  int64  
 12  EXTERNAL_SCORING_RATING_1     

In [186]:
#model = catb.CatBoostClassifier(**cb_params)
#scores = cross_val_score(model, train_data, train_target, cv=5, scoring='roc_auc')
#scores.mean()

In [187]:
x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=0.30)

cb_params = {
    "n_estimators": 500,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

model = catb.CatBoostClassifier(**cb_params)
model.fit(X = x_train, y = y_train, eval_set=[(x_valid, y_valid)])

0:	test: 0.6324468	best: 0.6324468 (0)	total: 37.7ms	remaining: 18.8s
10:	test: 0.6876459	best: 0.6876459 (10)	total: 385ms	remaining: 17.1s
20:	test: 0.6891858	best: 0.6907387 (19)	total: 1.08s	remaining: 24.6s
30:	test: 0.6947986	best: 0.6947986 (30)	total: 1.63s	remaining: 24.6s
40:	test: 0.6974270	best: 0.6977502 (36)	total: 2.05s	remaining: 23s
50:	test: 0.6990117	best: 0.6990117 (50)	total: 2.6s	remaining: 22.9s
60:	test: 0.7021776	best: 0.7021776 (60)	total: 3.31s	remaining: 23.9s
70:	test: 0.7030273	best: 0.7031483 (65)	total: 3.66s	remaining: 22.1s
80:	test: 0.7038377	best: 0.7038377 (80)	total: 4s	remaining: 20.7s
90:	test: 0.7041035	best: 0.7042004 (89)	total: 4.56s	remaining: 20.5s
100:	test: 0.7050378	best: 0.7050378 (100)	total: 5.13s	remaining: 20.3s
110:	test: 0.7059498	best: 0.7059498 (110)	total: 5.69s	remaining: 19.9s
120:	test: 0.7076418	best: 0.7076627 (119)	total: 6.37s	remaining: 20s
130:	test: 0.7085214	best: 0.7085214 (130)	total: 7.17s	remaining: 20.2s
140:	te

In [188]:
train_score = roc_auc_score(y_train, model.predict(x_train))
valid_score = roc_auc_score(y_valid, model.predict(x_valid))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)})")

Train-score: 0.506, Valid-score: 0.503)


In [189]:
y_pred_test = model.predict(test_data)
result_test = pd.DataFrame({
    "APPLICATION_NUMBER": test_data['APPLICATION_NUMBER'],
    "TARGET": y_pred_test.flatten(),})
result_test

,APPLICATION_NUMBER,TARGET
0,123724268,0
1,123456549,0
2,123428178,0
3,123619984,0
4,123671104,0
...,...,...
165136,123487967,0
165137,123536402,0
165138,123718238,0
165139,123631557,0


In [190]:
result_test['TARGET'].value_counts()

0    164965
1       176
Name: TARGET, dtype: int64

In [191]:
train_target.value_counts()

0    101196
1      8897
Name: TARGET, dtype: int64

In [192]:
filename = 'Vodorezova.csv'
result_test.to_csv(filename, index=None)